In [106]:
# ! pip install langchain langchain-google-genai langchain-openai langchain-community langchain dotenv langchain-text-splitters pypdf faiss-cpu langchain_classic

##### In this tutorial I'm using Google GEMINI MODEL & EMBEDDINGS but if you wanted to use OPENAI MODELS and EMBEDDINGS then please look at openai_rag.ipynb 

In [121]:
import os


In [ ]:
os.environ['GOOGLE_API_KEY'] = "API_KEY" # PASTE YOU'RE API KEY HERE.  
# set up you're api key - Google
# Get you're API key from https://aistudio.google.com/api-keys

#### Models: This is the "Brain." It handles the conversation and generates the final human-like response.

In [110]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                               temperature=.5 # Gemini 3.0+ defaults to 1.0
                               )

#### Embeddings: This is the "Translator." Computers don’t understand words; they understand numbers. Embeddings turn your text into long lists of numbers (vectors) so the computer can calculate how "similar" two sentences are.

In [111]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

#### Loaders: This is the "Data Entry" step. It simply opens your file ie.resume.pdf and reads the text so the code can process it. It gives you the documents. Suppose you're pdf has 5 pages then it gives you 5 Documents.

In [112]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('resume.pdf') # Document Loader
documents = loader.load()

#### Text-Splitters: Imagine trying to find a specific quote in a 500-page book. It’s easier if the book is broken into small, labeled snippets. We "chunk" the resume into 500-character pieces so the AI can pinpoint the exact section it needs without getting overwhelmed. Now our pdf of n pages broken down into chunks of 500 characters and each document contains 500 characters.  

In [113]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(documents)
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-24T10:47:32+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-24T10:47:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'resume.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Namilakonda Rahul rahulnamilakonda100@gmail.com\nGithub: github.com/rahulnamilakonda Hyderabad, India\nLinkedIn: linkedin.com/in/namilakondarahul/\nProfessional Summary\n• Junior Machine Learning Engineer with 2+ years of experience in Flutter development and hands-on skills in Python,\nFast API, SQL, NOSQL, GCP (CLOUD) and Data Preprocessing . Worked in a fast-paced startup environment at\n90Heal, contributing to AI-driven solutions for student mental health.\nExperience'),
 Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyp

#### Vector Store: It stores those numeric "translations" (embeddings) of your resume chunks. Because it's stored as numbers, the system can search through thousands of pages in milliseconds to find the most relevant information. You're query to this vector store is also converted to an embedding and performs a similarity search on stored embeddings to get the related documents to you're query.  

In [114]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(chunks, embedding=embeddings)

#### Retriever: This converts you're query to an embedding and performs a similarity search on all the stored vectors in vector store and gives you the related top 5 documents. 

In [115]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retriever.invoke("What is my experience")

[Document(id='444ea8d6-0af2-4674-ba6c-d08b0d3d6b4d', metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-24T10:47:32+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-24T10:47:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'resume.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='for personalized mental wellness recommendations using Gen AI, improving accuracy by 30%.\n◦ Cross-Platform App Development: Architected and developed a cross-platform mobile app using Flutter, connected\nto Firebase and GCP backend services, enabling personalized wellness tracking for 10,000+ students.\n◦ Applied Psychology and Behavioral Science : Integrated applied and behavioral psychology models within the'),
 Document(id='e9c5567d-44b9-4c65-8606-6c944945b33d', metadata={'producer': 'pdfTeX-1.40.26', 'c

#### Prompt

In [116]:
from langchain_core.prompts import ChatPromptTemplate


messages = [("system",
            """
            You are an AI Resume Assistant.

            Your task:
            - Answer questions about the candidate ONLY using the provided resume content.
            - Behave like a recruiter reviewing the resume.

            Guidelines:
            - Do not add skills, experience, or projects that are not explicitly mentioned.
            - If something is missing, clearly state that it is not present in the resume.
            - Prefer bullet points for skills, projects, and experience summaries.
            - Be factual, concise, and professional.

            Resume Context:
            {context}
            """
        ),
        ("human","""{input}""")
        ]

prompt = ChatPromptTemplate.from_messages(messages)


#### Output Parser: To parse the model output we use an output parser since the model response is very verbose and contains much information that an enduser don't require.

In [117]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

#### Connecting above all the compenents we now built the RAG system where we can ask questions on our PDF. 

In [118]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

combine_docs_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [119]:
# Questions: 
# Summarize the candidate's profile in 5 bullet points.

# What are the candidate's strongest technical skills?

# Does the candidate have backend development experience?

# Is the candidate suitable for a Data Scientist role?

# What kind of roles is this candidate best suited for?

# Give me top 5 interview questions which may be asked from my resume based on my working experience?

response = rag_chain.invoke({"input":"Give me top 5 interview questions which may be asked from my resume based on my working experience"})
response

{'input': 'Give me top 5 interview questions which may be asked from my resume based on my working experience',
 'context': [Document(id='3694901b-748c-4452-b5d8-253c11548365', metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-24T10:47:32+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-24T10:47:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'resume.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='application using Gen AI, Chain of Thought (CoT) reasoning, and Zero-shot prompting. Introduced audio-based chat\nfeatures for better user interaction.\n◦ Cloud Infrastructure and Data Pipelines : Optimized Google Cloud Platform (GCP) resources and BigQuery for\ndata analytics, reducing cloud costs by 25% and improving data efficiency.\n◦ Collaborative Leadership: Managed a team compris

In [120]:
response['answer']

'Based on your working experience at 90Heal, here are 5 interview questions that may be asked:\n\n1.  **Regarding the Gen AI RAG Pipeline:** "You led the integration of a Retrieval-Augmented Generation (RAG) pipeline for personalized mental wellness recommendations, improving accuracy by 30%. Can you elaborate on the architecture of this pipeline, the specific technologies you utilized (beyond Gen AI), and how you measured the 30% accuracy improvement?"\n2.  **Regarding Cloud Optimization:** "You optimized Google Cloud Platform (GCP) resources and BigQuery, reducing cloud costs by 25% and improving data efficiency. Can you describe the specific strategies, tools, or changes you implemented to achieve these significant cost savings and efficiency gains?"\n3.  **Regarding AI/ML Implementation:** "You implemented Chain of Thought (CoT) reasoning and Zero-shot prompting within the mental wellness application. Can you explain a specific scenario where you applied CoT or Zero-shot prompting 